# Creating Embredding file using Word2Vec

In [2]:
import re 
import pandas as pd 
from time import time 
from collections import defaultdict
import pandas as pd
import numpy as np

from gensim.models import Word2Vec
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk import pos_tag
from nltk.corpus import wordnet
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import spacy

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

C:\Users\miman\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\miman\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\miman\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\miman\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

Using the below 2 files to create embeddings

In [3]:
questions = pd.read_csv('Questions_Answers_126Products.csv')
reviews = pd.read_csv('Reviews_126Products.csv')

In [4]:
questions.head()

,Prod_ID,Ques_ID,Question Text,Ans_ID,Ans_Text,Q_A
0,1,0,Will this keep a strong 6 year old out of my c...,0,"Yes, the locks will keep adults out too. My h...",Will this keep a strong 6 year old out of my c...
1,1,0,Will this keep a strong 6 year old out of my c...,1,Yes if you install it correctly. a lot of fol...,Will this keep a strong 6 year old out of my c...
2,1,0,Will this keep a strong 6 year old out of my c...,2,It probably will... it's pretty good and much...,Will this keep a strong 6 year old out of my c...
3,1,1,What's the difference between this and the reg...,3,The size of the locking mechanism. I bought th...,What's the difference between this and the reg...
4,1,1,What's the difference between this and the reg...,4,The locking mechanism unlocks with the magnet .,What's the difference between this and the reg...


In [5]:
reviews.head()

,Review_ID,Product ID,Review_Text
0,0,1,'not really as portrayed on video
1,1,1,'HORRIBLE installation experience. Installed p...
2,2,1,'I wish we'd bought the cheaper \'push latch\'...
3,3,1,"'The concept is great, but these things are a ..."
4,4,1,'My husband installed 4 of these locks on my s...


In [6]:
from string import punctuation
punctuation = list(punctuation)
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in punctuation:
           # pos = pos_tag([i.strip()])
            word = lemmatizer.lemmatize(i.strip())
            final_text.append(word.lower())
    return " ".join(final_text)

In [7]:
reviews = reviews.dropna()
questions = questions.dropna()
questions['Q'] = questions.Q_A.apply(lemmatize_words)
reviews['R'] = reviews.Review_Text.apply(lemmatize_words)
import re,string,unicodedata
questions.Q = [re.sub(r'\d+', '', x) for x in questions.Q]
reviews.R = [re.sub(r'\d+', '', x) for x in reviews.R]

In [8]:
all_sentences = pd.Series()
all_sentences=all_sentences.append(questions.Q)
all_sentences=all_sentences.append(reviews.R)
len(all_sentences)

C:\Users\miman\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


30304

In [8]:
all_sen = []
for i in all_sentences:
    all_sen.append(i.split())

flat_all_sen = []
for sublist in all_sen:
    for item in sublist:
        if item!=-1 or item:
            flat_all_sen.append(item)

#### Creating Word2Vec model

In [13]:
w2v_model = Word2Vec(min_count=1,size=100,)

In [14]:
t = time()
w2v_model.build_vocab(all_sen)
print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

INFO - 02:10:38: collecting all words and their counts
INFO - 02:10:38: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 02:10:38: PROGRESS: at sentence #10000, processed 800902 words, keeping 33322 word types
INFO - 02:10:38: PROGRESS: at sentence #20000, processed 1640754 words, keeping 55448 word types
INFO - 02:10:38: PROGRESS: at sentence #30000, processed 2494651 words, keeping 74401 word types
INFO - 02:10:38: collected 74916 word types from a corpus of 2521765 raw words and 30304 sentences
INFO - 02:10:38: Loading a fresh vocabulary
INFO - 02:10:38: effective_min_count=1 retains 74916 unique words (100% of original 74916, drops 0)
INFO - 02:10:38: effective_min_count=1 leaves 2521765 word corpus (100% of original 2521765, drops 0)
INFO - 02:10:39: deleting the raw counts dictionary of 74916 items
INFO - 02:10:39: sample=0.001 downsamples 52 most-common words
INFO - 02:10:39: downsampling leaves estimated 1873167 word corpus (74.3% of prior 2521765)
INFO 

Time to build vocab: 0.2 mins


In [15]:
w2v_model.corpus_count

30304

In [16]:
t = time()

w2v_model.train(all_sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

INFO - 02:11:06: training model with 3 workers on 74916 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
INFO - 02:11:07: EPOCH 1 - PROGRESS: at 25.80% examples, 2137593 words/s, in_qsize 5, out_qsize 0
INFO - 02:11:08: EPOCH 1 - PROGRESS: at 46.17% examples, 2141970 words/s, in_qsize 6, out_qsize 1
INFO - 02:11:09: EPOCH 1 - PROGRESS: at 70.60% examples, 2188896 words/s, in_qsize 5, out_qsize 0
INFO - 02:11:10: EPOCH 1 - PROGRESS: at 95.48% examples, 2210994 words/s, in_qsize 5, out_qsize 0
INFO - 02:11:10: worker thread finished; awaiting finish of 2 more threads
INFO - 02:11:10: worker thread finished; awaiting finish of 1 more threads
INFO - 02:11:10: worker thread finished; awaiting finish of 0 more threads
INFO - 02:11:10: EPOCH - 1 : training on 13081757 raw words (9375012 effective words) took 4.2s, 2212846 effective words/s
INFO - 02:11:11: EPOCH 2 - PROGRESS: at 26.87% examples, 2247671 words/s, in_qsize 5, out_qsize 0
INFO - 02:11:12: EPOCH 2 - P

INFO - 02:11:53: EPOCH 12 - PROGRESS: at 49.59% examples, 2379635 words/s, in_qsize 5, out_qsize 0
INFO - 02:11:54: EPOCH 12 - PROGRESS: at 75.74% examples, 2380304 words/s, in_qsize 5, out_qsize 0
INFO - 02:11:55: worker thread finished; awaiting finish of 2 more threads
INFO - 02:11:55: worker thread finished; awaiting finish of 1 more threads
INFO - 02:11:55: worker thread finished; awaiting finish of 0 more threads
INFO - 02:11:55: EPOCH - 12 : training on 13081757 raw words (9374315 effective words) took 3.9s, 2382111 effective words/s
INFO - 02:11:56: EPOCH 13 - PROGRESS: at 27.62% examples, 2302956 words/s, in_qsize 5, out_qsize 0
INFO - 02:11:57: EPOCH 13 - PROGRESS: at 48.54% examples, 2317277 words/s, in_qsize 5, out_qsize 0
INFO - 02:11:58: EPOCH 13 - PROGRESS: at 73.71% examples, 2318407 words/s, in_qsize 5, out_qsize 0
INFO - 02:11:59: EPOCH 13 - PROGRESS: at 99.16% examples, 2321304 words/s, in_qsize 5, out_qsize 0
INFO - 02:11:59: worker thread finished; awaiting finish 

INFO - 02:12:40: worker thread finished; awaiting finish of 0 more threads
INFO - 02:12:40: EPOCH - 23 : training on 13081757 raw words (9374839 effective words) took 4.1s, 2260027 effective words/s
INFO - 02:12:41: EPOCH 24 - PROGRESS: at 28.14% examples, 2357956 words/s, in_qsize 5, out_qsize 0
INFO - 02:12:42: EPOCH 24 - PROGRESS: at 49.39% examples, 2365849 words/s, in_qsize 5, out_qsize 0
INFO - 02:12:43: EPOCH 24 - PROGRESS: at 75.29% examples, 2371603 words/s, in_qsize 5, out_qsize 0
INFO - 02:12:44: worker thread finished; awaiting finish of 2 more threads
INFO - 02:12:44: worker thread finished; awaiting finish of 1 more threads
INFO - 02:12:44: worker thread finished; awaiting finish of 0 more threads
INFO - 02:12:44: EPOCH - 24 : training on 13081757 raw words (9374023 effective words) took 4.0s, 2373063 effective words/s
INFO - 02:12:45: EPOCH 25 - PROGRESS: at 27.36% examples, 2285562 words/s, in_qsize 5, out_qsize 0
INFO - 02:12:46: EPOCH 25 - PROGRESS: at 47.69% examples

Time to train the model: 2.03 mins


In [17]:
words = list(w2v_model.wv.vocab)

In [26]:
sent = ' '.join(word for word in words)

In [27]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(list(words))
word_index = tokenizer.word_index

In [28]:
vocab = pd.read_csv('Vocab_file.csv')
list_vocab = list(vocab['Unnamed: 0'])

In [29]:
len(list_vocab)

27150

In [30]:
(w2v_model['baby']).shape

C:\Users\miman\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


(100,)

In [35]:
count = 0
skip = 0
a_file = open("Embedding_100dim.txt", "w")
for word in list_vocab:
    try:
        np.savetxt(a_file, w2v_model[word].reshape(1,100))
    except KeyError:
        skip = skip + 1
        np.savetxt(a_file, w2v_model['baby'].reshape(1,100))
    count = count +1
print(count)
a_file.close()

C:\Users\miman\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\miman\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


27150
